In [1]:
#Importing all desired libraries for scraping website data along with libraries for data arrangement

import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup, SoupStrainer
import ssl
import requests
import re 
import random
from itertools import groupby
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
#Bypassing Bott Detection 
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [3]:
# Extracting the top 5 webpages from the website
page_urls = []
for i in range(5):
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page='+ str(i)
# Extracted all the web pages from which we have to extract movie data
    page_urls.append(url)
#print(page_urls)

In [4]:
# Creating empty lists to save movie data and movie URL's
my_movies = []
page_link = []
#Iterating over the list of page links
for i in page_urls:
#Saving the HTML content from the webpage in an html object
    html = requests.get(i,headers = headers)
#Parsing all the HTML data and savign the same in a soup object using beautifulsoup
    soup = BeautifulSoup(html.content, "html.parser")
# Searcing for the tag which consists of the movies name and using find_all function to save all the desired data in as a list
#Using find_all rerurns data as a list
    container = soup.find_all('td',class_ = "clamp-summary-wrap")  
#Iterating over the soup object to extract movies names
    for i in container:
#Since the webiste content comes with a lot of tage we are removing all anchor tags and just extracting the text information that we need
        movie_tag = i.find('h3')
        movie_name = movie_tag.text
#Appending each movie name to a list
        my_movies.append(movie_name.strip())
#Extracting individual movie URL and saving in a list
        #link_tag = i.find('href')
        my_link = i.find('a',class_='title')
        link = my_link.get('href',None)
        page_link.append(link.strip())

In [5]:
print(len(page_link))

500


In [6]:
#For each url extracted and stored in the list page_link.
# Creating a complete accessable link for each movie in the list
for i in range(len(page_link)):
    page_link[i] = 'https://www.metacritic.com' + page_link[i]
#print(page_link)
print(len(page_link))

500


In [7]:
#Creating a list to store the detail page URL/link for each movie in page_link
detail_links = []
for i in page_link:
#Making another connection to the website using requests to access the webpage
    details = requests.get(i,headers = headers)
#Saving Parsed HTML in a new container
    detail_page = BeautifulSoup(details.content, 'html.parser')
#The information we want to extract is of type /movie/citizen-kane-1941/details and it is stored under the div tag.
# The class_ information specified here is basically the unique table attribute under which all the relevant data is stored
    detail_container = detail_page.find_all('div',class_='see_all pad_btm1')
#Iterating over all the div class objects to extract (/movie/citizen-kane-1941/details) which is saved under an anchor tag
    for i in detail_container:
        full_link = i.find('a').get('href',None)
        #print(full_link)
#Appending the urls in a list called detail_links
        detail_links.append(full_link.strip())
print(len(detail_links))

492


In [8]:
#In the earlier step we extracted just the links that will redirect you to the movie page (https://www.metacritic.com/movie/casablanca_)
#Here we are repeating the same process to get link which will redirect you to the detail page of the movie
for i in range(len(detail_links)):
    detail_links[i]= 'https://www.metacritic.com'+detail_links[i].strip()
print(len(detail_links))


492


# Extracting Director Names

In [9]:
# I have extracted all the relevent movies names and detailed information page link for each movie
#In this cell I am extracting the director(s) who have directed all the movies in my list of movies
# I have created 3 lists here out of which all_director is my list of directors and the other 2 lists are for temporary usage

all_director=[]
director2 = []
empty_value = ['']
for i in detail_links:
    cast_link = requests.get(i,headers = headers)
    cast_page = BeautifulSoup(cast_link.content,'html.parser')
#Here I am extracting table where all the director names are placed on the website
#As we are extracting information from over 500 pages its natural that the processing time will be very high
# I am using a regular ecpression function called compile that helps me with the slowness
# So instead iterating over 500 movies names to get the Director Credits , my code is taking 'Director Credits' as a generic seach argument and extraction data for all the movies instead
# As a result I am just iterating over different movie links and storing that information in list object called director_table   
    director_table = cast_page.find_all('table', attrs = {'summary': re.compile('Director Credits')})
#I noticed a lot of blank spaces and NONE type objects while iterating over director_table and hence I am taking cases where my object lenth is not 0
    if len(director_table) != 0:
        for directors in director_table:
# Extracting director information that is stored under the a tag
            dt = directors.find_all('a')
# Here I am checking the len of directors, so for movies that are directed by a single director will be considered here.
            if len(dt) == 1:
# Iterating over each line that contains the Director Name
                for k in dt: 
#Extracting just the director name and appending them to my main list
                    all_director.append(k.text.strip())
#Cases where a movie has several directors are taken care of here
            else:   
                for each in dt:
# Here , I am storing the multiple director information in a temporary list called director2 
                    director2.append(each.text.strip())
# Now I am appending my temp list to my main list and I am emptying it to store fresh multiple director details
                all_director.append(director2)
                director2 = []
        
# Appending an empty value to my main list if there are any blank spaces or if any movie does not have a director/credit/cast
    else:
        all_director.append(empty_value)

In [10]:
print(all_director)

['Francis Ford Coppola', 'Orson Welles', 'Alfred Hitchcock', 'Michael Curtiz', 'Richard Linklater', 'Krzysztof Kieslowski', 'Alfred Hitchcock', 'Alfred Hitchcock', ['Gene Kelly', 'Stanley Donen'], 'Charles Chaplin', 'Barry Jenkins', 'D.W. Griffith', ['Ben Sharpsteen', 'Bill Roberts', 'Hamilton Luske', 'Jack Kinney', 'Norman Ferguson', 'T. Hee', 'Wilfred Jackson'], 'Orson Welles', 'Alfred Hitchcock', 'John Huston', 'Guillermo del Toro', 'Billy Wilder', 'Alfred Hitchcock', 'Steve James', 'Akira Kurosawa', 'Joseph L. Mankiewicz', 'François Truffaut', 'Sam Peckinpah', 'Jim Sheridan', 'Carol Reed', 'Stanley Kubrick', 'Jasmila Zbanic', ['George Cukor', 'Sam Wood', 'Victor Fleming'], 'Cristian Mungiu', 'Alfred Hitchcock', 'Sergei M. Eisenstein', 'Elia Kazan', 'John Huston', 'George Lucas', ['Ben Sharpsteen', 'Bill Roberts', 'Jack Kinney', 'John Elliotte', 'Norman Ferguson', 'Samuel Armstrong', 'Wilfred Jackson'], 'Alfonso Cuarón', 'Akira Kurosawa', 'Ahmir-Khalib Thompson', 'Ernst Lubitsch', '

# EXTRACTING CAST

In [11]:
#################EXTRACTING CAST ######################################################################
#In this cell I am extracting information on cast members of all 500 movies in my list
# I have created 3 lists here out of which all_cast is my list of cast members and the other 2 lists are for temporary usage
# The process to extract the director and the cast information is the same. For extracting any such information we need to acces the webpage, parse the html content
# We need to pick up the relevant tags were our information is stored and then we push all the information in a list
p_c_temp = []
all_cast = []
empty = ['']
for i in detail_links:
        cast_link = requests.get(i,headers = headers)
        cast_page = BeautifulSoup(cast_link.content,'html.parser')
#Saving all the principal cast and cast information under p_cast object
#The re.complie function is handeling cases and will take both cases wehre summary is stored for Principal Cast and Normal Cast
        p_cast = cast_page.find_all('table', attrs = {'summary': re.compile('Cast Credits')})
#Checking over those elements of list obeject that are non zero
        if len(p_cast) != 0:
# Here each we iterate over is a line contaning the name of the cast member
            for each in p_cast:
#Cast member and principal cast info are found under 'a' tag hence I am using find_all to create a list object
# My list object contains lines that include cast member names
                ml = each.find_all('a')
# AS there are certain unwanted tags and html data in my list object, I am extracting just the text information and i am checking for whitespaces using strip
                for x in ml:
# Appending all cast and principal cast into a temp list first
                    p_c_temp.append(x.text.strip())
# Appending the temp list to the main list
#My main list is a collection of 500 nested lists.
#Each nested list contains all the cast information of a distinct movie
            all_cast.append(p_c_temp)
            p_c_temp = []
# Appending an empty value to my main list if there are any blank spaces or if any movie does not have a director/credit/cast
        else:
            all_cast.append(empty)

# Extracting Movie Credits

In [12]:
#################EXTRACTING Credits ######################################################################
#In this cell I am extracting the cast credits information. The roles each cast member has played in a movie
#I have created 3 lists here out of which all_credits is my main list of cast credits and the other 2 lists are for temporary usage and for storing blank spaces
 #The process to extract the credit information is same as the other tables I have extracted above. The only difference here is that I am a td tag here instead of anchor tag (a)
# We need to pick up the relevant tags were our information is stored and then we push all the information in a list
all_credits = []
credits_temp = []
empty = ['']
for i in detail_links:
        cast_link = requests.get(i,headers = headers)
        cast_page = BeautifulSoup(cast_link.content,'html.parser')
#Using beautifulsoup I am parsing the HTML content and I am creating a list object called cast_credits
# This object now contains only relevant information that needs to be extracted
        cast_credits = cast_page.find_all('table', attrs = {'summary': re.compile('Cast Credits')})
#Checking over those elements of list obeject that are non zero
        if len(p_cast) != 0:
# Here each we iterate over is a line contaning the cast credit information
            for a in cast_credits:
#Credits info are found under the 'td' tag hence I am using find_all to create a list object
# My list object contains lines that include credit information
                x = a.find_all('td',class_ = 'role')
# AS there are certain unwanted tags in my list object, I am extracting just the text information and i am checking for whitespaces using strip
                for line in x:
# Appending all creditrs info to the temp list
                    credits_temp.append(line.text.strip())
 #Appending the temp list to the main list
#My main list is a collection of 500 nested lists.
#Each nested list contains all the credit information of a distinct movie
            all_credits.append(credits_temp)
            credits_temp = []
# Appending an empty value to my main list if there are any blank spaces or if any movie does not have a director/credit/cast
        else:
            all_credits.append(empty)

# List Conversion to Tuple

In [13]:
# Converting director list to a tuple
directors = []
for each in all_director:
    if type(each) == list:
        directors.append(tuple(each))
    else:
        directors.append(each)
directors = tuple(directors)

#Converting cast list to tuple
casts = []
for each in all_cast:
    casts.append(tuple(each))
print(len(casts))

492


# Creating a Combined Dictonary

In [14]:
#In this block I am taking all my movie related information (cast,credits,director) and I am creading a combined dictonary
# Creating my holistic dictonary
movie_details = {}
# Since I have converted my lists to tuple, I need a iterator to iterate over my tuple object perfectly
# For this I have used zip function. zip groups lines from all the 3 obejects I have used in every iteration
# So instead of calling 3 different list, I just call the zip object.
allzip = zip(my_movies,directors,casts)
for i,j,k in allzip:
#In the list of movies, there are cases where the movie name is exactly the same.
#This creates a problem when we are creating a dictonary with movie name as the key as we cant have duplicate keys in a dict
# To couter this I am appending a random generated number against duplicate movie names
    if i in movie_details:
        z = str((random.randint(0,9))) + i
        movie_details[z] = {j:k}
    else:
        movie_details[i] = {j:k}
# Here j is director and k is cast

In [15]:
movie_details

{'The Godfather': {'Francis Ford Coppola': ('Al Pacino',
   'Marlon Brando',
   'Abe Vigoda',
   'Al Lettieri',
   'Diane Keaton',
   'Gianni Russo',
   'James Caan',
   'John Cazale',
   'John Marley',
   'Richard Conte',
   'Richard S. Castellano',
   'Robert Duvall',
   'Rudy Bond',
   'Sterling Hayden',
   'Talia Shire')},
 'Citizen Kane': {'Orson Welles': ('Joseph Cotten',
   'Orson Welles',
   'Agnes Moorehead',
   'Dorothy Comingore',
   'Erskine Sanford',
   'Everett Sloane',
   'Fortunio Bonanova',
   'George Coulouris',
   'Georgia Backus',
   'Gus Schilling',
   'Harry Shannon',
   'Paul Stewart',
   'Philip Van Zandt',
   'Ray Collins',
   'Ruth Warrick',
   'William Alland')},
 'Rear Window': {'Alfred Hitchcock': ('Frank Cady',
   'Georgine Darcy',
   'Grace Kelly',
   'James Stewart',
   'Judith Evelyn',
   'Raymond Burr',
   'Ross Bagdasarian',
   'Sara Berner',
   'Thelma Ritter',
   'Wendell Corey',
   'Havis Davenport',
   'Irene Winston',
   'Jerry Antes',
   'Jessly

# Mapping Movie and Credits

In [16]:
#MOVIE CAST CREDIT MAPPING
# Creating a mapping of movies and credits
#Storing the saem in a dictonary
movie_credits = {}
zip_object_3 = zip(my_movies,all_credits)
for i,j in zip_object_3:
    if i in movie_credits:
        c = str((random.randint(0,9))) + i
        movie_credits[c] = j
    else:
        movie_credits[i] = j

# Check movie related information below!






In [17]:
#Using a try and except blcok to catch any errors due to invalid input
# There are 2 inputs I am asking for
#Input one asks you to enter a movie name
#Input 2 Asks you to enter 'cast' or 'director'
try:
    print('MOVIE OPTIONS!',[i for i in movie_details.keys()])
    i1 = input('Which Movie do you want to check?: ')
    i2 = input('What information about this movie do you want to check? (Choose director or cast): ')
#If user inputs 'cast' as the second input    
    if i2 == 'cast':
#Storing all the values (directors,casts) for the movie name that user entered
        m = movie_details[i1]
# Since the data stored in m is also a dict object
# I am iterating over the values
#Keys are director names and values are cast
        for cast in m.values():
# IF a movie does not have any cast, the following will be printed
            if cast == ('',):
                print('Looks Like This movies has no cast member. What a bummer!')
            else:
# I am saving info in a variable x
                x = cast
# I am storing the credit info the movie in a variable z
        z = movie_credits[i1]
# I am zipping my tuple objects
        my_zip = zip(x,z)
#Iterating over the tuple to extract grouped values
        for a,b in my_zip:
# Printing cast and the credits they have played
            print(a ,'as', b)
#If second user input is 'director'
    elif i2 == 'director':
# I am storing the 'director' information which is stored as a value for the movie name entered by the user
        d = movie_details[i1]
#Iterating over the dictonary of direct and cast and extracting only the director info
        for director in d.keys():
# Saving director info in a cariable and printing the same
            a= director
        print('The movie is directed by: ',str(a))
    else:
        print('Oops! Something is wrong with your input. Please try again!')
except:
    print('Invalid Movie Name OR the second input is wrong. Please try again!')
        

MOVIE OPTIONS! ['The Godfather', 'Citizen Kane', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Lady Vanishes', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Quo Vadis, Aida?', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'The Maltese Falcon', 'American Graffiti', 'Dumbo', 'Roma', 'Parasite', 'Ran', 'Summer of Soul (...Or, When the Revolution Could Not Be Televised)', 'The Shop Around the Corner', '12 Angry Men', "Rosemary's Baby", '12 Years a Slave', 'Manchester by the Sea', 'Killer of Sheep', 'Nashville', 'Ratatouille', 'The Grapes of Wrath', 'Chi

Which Movie do you want to check?: The Godfather
What information about this movie do you want to check? (Choose director or cast): director
The movie is directed by:  Francis Ford Coppola


# Importing Data to CSV

In [18]:
#Importing my data to a csv file
import csv
from itertools import zip_longest
lists = [my_movies,all_director,casts]
export_data = zip_longest(*lists, fillvalue = '')
with open('saimmohammd_finalproject.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("Movies", "Director","Cast"))
    wr.writerows(export_data)
myfile.close()

# Question 1 (a)

# Movies Directed by each Director

In [19]:
#### Direcor Count #######
# creating an empty dictonary to store director as keys and number of movies directed as count
director_count = {}
# Accessing dictonary values ( a nested dictonary of director and cast for each movie)
for directors in movie_details.values():
#Taking the directors from the dictonary
    for director in directors.keys():
# Checking for cases with multiple directors
        if type(director) == tuple:
# Iterating over multiple directors
            for each in director:
#Assigning a value of 1 if the director is not in the dictonary
                if each not in director_count:
                    director_count[each] = 1
                else:
#Incrementing the director value if he/she aklready exists by 1
                    director_count[each] = director_count[each] + 1
# Condition where there are single directors for a movie
        elif director not in director_count:
            director_count[director] = 1
        else:
            director_count[director] = director_count[director] + 1
#Sorting the dict
# Importing operator liab to sort my data in order as saving it as a list
import operator
director_count_sorted = sorted(director_count.items(), key=operator.itemgetter(1),reverse = True)
#print(director_count_sorted)

In [20]:
print(director_count_sorted)

[('Alfred Hitchcock', 12), ('', 10), ('Martin Scorsese', 8), ('Francis Ford Coppola', 6), ('John Ford', 6), ('Orson Welles', 5), ('John Huston', 5), ('Akira Kurosawa', 5), ('Stanley Kubrick', 5), ('George Cukor', 5), ('Hirokazu Koreeda', 5), ('Michael Curtiz', 4), ('Ben Sharpsteen', 4), ('Bill Roberts', 4), ('Wilfred Jackson', 4), ('Robert Altman', 4), ('Mike Leigh', 4), ('Peter Jackson', 4), ('Spike Lee', 4), ('Howard Hawks', 4), ('Steven Spielberg', 4), ('Norman Ferguson', 3), ('Billy Wilder', 3), ('Steve James', 3), ('Samuel Armstrong', 3), ('Alfonso Cuarón', 3), ('David Hand', 3), ('James Whale', 3), ('Leo McCarey', 3), ('Joel Coen', 3), ('Paul Thomas Anderson', 3), ('David Lean', 3), ('Lee Unkrich', 3), ('Apichatpong Weerasethakul', 3), ('Frederick Wiseman', 3), ('Jean-Pierre Dardenne', 3), ('Luc Dardenne', 3), ('Pedro Almodóvar', 3), ('Richard Linklater', 2), ('Krzysztof Kieslowski', 2), ('Barry Jenkins', 2), ('Hamilton Luske', 2), ('Jack Kinney', 2), ('T. Hee', 2), ('Guillermo d

# Question 1 (b)

# Times a cast member appeared in the top 500 movies

In [21]:
##### CAST COUNT ############
#creating an empty dictonary to store cast count
cast_count = {}
# Iterating over the dict value(value is a nested dict)
for cast_details in movie_details.values():
# Iterating over nested dict(value is cast members)
    for cast in cast_details.values():
        for each in cast:
# Adding cast memeber to the dict I created and assigning a value of 1
            if each not in cast_count:
                cast_count[each] = 1
# Incrementing the values if cast member already present
            else:
                cast_count[each] = cast_count[each] + 1
# Importing operator liab to sort my data in order as saving it as a list
cast_count_sorted = sorted(cast_count.items(), key=operator.itemgetter(1),reverse = True)
print(cast_count_sorted)

[('', 19), ('Harvey Keitel', 8), ('Robert De Niro', 8), ('Marlon Brando', 7), ('John Ratzenberger', 7), ('Timothy Spall', 7), ('Jim Broadbent', 7), ('Robert Duvall', 6), ('Orson Welles', 6), ('James Stewart', 6), ('Ward Bond', 6), ('Harrison Ford', 6), ('Lucas Hedges', 6), ('Lesley Manville', 6), ('Saoirse Ronan', 6), ('Willem Dafoe', 6), ('Fabrizio Rongione', 6), ('Joseph Cotten', 5), ('Humphrey Bogart', 5), ('John Qualen', 5), ('Ethan Hawke', 5), ('Daniel Day-Lewis', 5), ('Thomas Mitchell', 5), ('Kyle Chandler', 5), ('Caleb Landry Jones', 5), ('Rooney Mara', 5), ('Ralph Fiennes', 5), ('Cate Blanchett', 5), ('Andy Serkis', 5), ('Emma Stone', 5), ('Frances McDormand', 5), ('Michael Keaton', 5), ('Tilda Swinton', 5), ('Richard Ayoade', 5), ('Bradley Cooper', 5), ('Al Pacino', 4), ('Rudy Bond', 4), ('Sterling Hayden', 4), ('Ray Collins', 4), ('Cary Grant', 4), ('Mahershala Ali', 4), ('Takashi Shimura', 4), ('George C. Scott', 4), ('Karl Malden', 4), ('Murray Alper', 4), ('Henry Fonda', 4

# Question 1 (c)

# Number of times a Director  acted in a movie 

In [22]:
############ Director in Cast ###################
# Creating a dict to store the subjected info
dir_in_cast = {}
# Iterating over the values of my dict ( Value is a nested dict of director and cast)
for cast_details in movie_details.values():
# Checking for cast members in the sub dict
    for cast in cast_details.values():
        for each in cast:
# Iterating over the director table 
            for j in director_count:
# Checking if cast member is in director table
                if j == each:
                    dir_in_cast[j] = dir_in_cast.get(j,0)+1
                    
# Importing operator liab to sort my data in order as saving it as a list
dir_in_cast_sorted = sorted(dir_in_cast.items(), key=operator.itemgetter(1),reverse = True)
#print(dir_in_cast_sorted)

In [23]:
#Storing director and cast info in a list as a dictonary
#I wanted a director to cast mapping here
#I am taking values (director,cast) from my main dict movie_details
#I am appending the value elemet in a list 
#This way I still maintain a dict with a list
directorcast = []
for a in movie_details.values():
    directorcast.append(a)
#print(directorcast)

In [24]:
# Flattening the director cast mapping
# To format my data further, I am flattening my list of dictonaries
# I have rearranged it such a way that if multiple directors have directed a single movie, each director:cast will be appended seperately
# Current Strutcure [{('director1','director2'):(cast members)}]
#Converted Strutcure - [{('director1'):(cast members)},{('director2'):(cast members)}]
#Created a new list to append my data into
flat_dir_cast  = []
# Iterating over castdirector list from previous block
for information in directorcast:
#Accessing the key and value of the dictonary element in the list
    for director,cast in information.items():
#Cheking for conditions where there are multiple directors
        if type(director) == tuple:
            for each in director:
#Appending each director and cast seperately
                flat_dir_cast.append({each:cast})
#Cases where there is only a single director
        else:
            flat_dir_cast.append(information)


# Question 2 ------- Calculating Cosine Similarity
# Part 1 - Taking User Inputs

In [25]:
###################### QUESTION 2###################################################
# Here I am asking user to input any 2 director names tthat the user wants to compare
print('Choose any Director Names from: ',[i for i in director_count.keys()]) 
inp1 = input('Director1: ')
inp2 = input('Director2: ')
# I am cecking if the input1 exists in my director repository
if inp1 in director_count.keys():
# Initializing an empty list to store all the cast members who have worked with a director
    dir_cast_1 = []
# Iterating over the flat_dict i have created in the above cell 
    for data in flat_dir_cast:
#Iterating over the key and value of my the dict
        for director,val in data.items():
# If the user inpit matches the director in my flat dictonary I am appending all the cast members who have worked with that director
            if inp1 == director:
                dir_cast_1.extend(val)
else:
    print('Director 1 name not found')
            
            
#Now I am checking the frequency of these cast members, how many times they have appeared in a movie with a director
#Creating a dictonary to store cast name and number of times they worked with a director
dir_cast_count = {}
# Iterating over my list of cast members created above
for stars in dir_cast_1:
# If cast are not present then I am assigning a value of 1
    if stars not in dir_cast_count:
        dir_cast_count[stars] = 1
#If cast already present then the value is incremented by 1
    else:
        dir_cast_count[stars] += 1
        
        
        
# Same procedure as above for the second input        
 ################SECOND DIRECTOR#####################################
if inp2 in director_count.keys():
    dir_cast_2 = []
    for data in flat_dir_cast:
        for director,val in data.items():
            if inp2 in director:
                dir_cast_2.extend(val)
else:
    print('Director 2 Key not found')

dir_cast_count_2 = {}
for stars in dir_cast_2:
    if stars not in dir_cast_count_2:
        dir_cast_count_2[stars] = 1
    else:
        dir_cast_count_2[stars] += 1

Choose any Director Names from:  ['Francis Ford Coppola', 'Orson Welles', 'Alfred Hitchcock', 'Michael Curtiz', 'Richard Linklater', 'Krzysztof Kieslowski', 'Gene Kelly', 'Stanley Donen', 'Charles Chaplin', 'Barry Jenkins', 'D.W. Griffith', 'Ben Sharpsteen', 'Bill Roberts', 'Hamilton Luske', 'Jack Kinney', 'Norman Ferguson', 'T. Hee', 'Wilfred Jackson', 'John Huston', 'Guillermo del Toro', 'Billy Wilder', 'Steve James', 'Akira Kurosawa', 'Joseph L. Mankiewicz', 'François Truffaut', 'Sam Peckinpah', 'Jim Sheridan', 'Carol Reed', 'Stanley Kubrick', 'Jasmila Zbanic', 'George Cukor', 'Sam Wood', 'Victor Fleming', 'Cristian Mungiu', 'Sergei M. Eisenstein', 'Elia Kazan', 'George Lucas', 'John Elliotte', 'Samuel Armstrong', 'Alfonso Cuarón', 'Ahmir-Khalib Thompson', 'Ernst Lubitsch', 'Sidney Lumet', 'Roman Polanski', 'Kenneth Lonergan', 'Charles Burnett', 'Robert Altman', 'Brad Bird', 'John Ford', 'Marcel Carné', 'Martin Scorsese', 'Kenji Mizoguchi', 'Preston Sturges', 'David Hand', 'Ford Bee

In [26]:
#combined list containing all actors that the 2 directors have worked with
#first we extract the unique actor names from the dictonary created in the above cell
cast1 = list(dir_cast_count.keys())
cast2 = list(dir_cast_count_2.keys())
#print(cast1)
#print(cast2)

#Converting each of the lists above into a set
# This is done for merging and applying union on 2 set of values
cast_set_1 = set(cast1)
cast_set_2 = set(cast2)

#The union set
combined_cast = cast_set_1.union(cast_set_2)

# Creating 2 dictonaries with initial value of 0 
dir_actor_1 = dict()
dir_actor_2 = dict()
# Giving a value of 0 to all keys
for i in combined_cast:
    dir_actor_1[i] = 0
    dir_actor_2[i] = 0

#Here I am iterating over the items of 2 dictonaries 
# if key from dictonary 1 matches key in dictonary 2
#I am appending the value from dictonary 2 to value dictonary against the common keys
for name, val in dir_actor_1.items():
    for cast, value in dir_cast_count.items():
        if name == cast:
            dir_actor_1[name] = value
            
            
#Here I am iterating over the items of 2 dictonaries 
# if key from dictonary 1 matches key in dictonary 2
#I am appending the value from dictonary 2 to value dictonary against the common keys
for name, val in dir_actor_2.items():
    for cast, value in dir_cast_count_2.items():
        if name == cast:
            dir_actor_2[name] = value
#I have taken all the numeric values from the dictonaries and have saved them as lists
dir_vector_1 = list(dir_actor_1.values())
dir_vector_2 = list(dir_actor_2.values())


#calculating cosine similarity
#Variable initialization
similar_score_divident = 0 
for i in range(len(dir_vector_1)):
# Multiplying each element of numeric list created above with the second list elements 
    total = dir_vector_1[i] * dir_vector_2[i]
    similar_score_divident = similar_score_divident + total
#print(similar_score_divident)

#Initializing variables with a value of 0 
x = 0
y = 0
#Since the lenths of both lists are equal,I am taking the everything in the range of dir_vector_1
for i in range(len(dir_vector_1)):
#Calculating the total of dir_vector_1
    total_1 = (dir_vector_1[i])**2
    x = x + total_1
#Calculating the total of dir_vector_2
    total_2 = (dir_vector_2[i])**2
    y = y + total_2
score_divisor = (x**0.5)*(y**0.5)
#The final Cosine Similarity Calculation
score = similar_score_divident/score_divisor
print('Cosine Similarity is: ', score)

Cosine Similarity is:  0.0


# Question 3

Chosen Cast Members are - 
Al Pacino, Christian Bale, Robert De Niro, Brad Pitt, Edward Norton

In [27]:
# Asking user to input 5 of there factor actors
print("Choose cast members for whom you want to calculate the similary score: ")
c1 = input("Cast Member 1 : ")
c2 = input("Cast Member 2 : ")
c3 = input("Cast Member 3 : ")
c4 = input("Cast Member 4 : ")
c5 = input("Cast Member 5 : ")
    
#Finding all the other cast members our first actor has worked with
if c1 in cast_count.keys() and c2 in cast_count.keys() and c3 in cast_count.keys() and c4 in cast_count.keys() and c5 in cast_count.keys():
    c1_cast = []
    for i in directorcast:
        for j in i.values():
            if c1 in j:
#Appending all the related castmembers
                c1_cast.extend(j)
#Removing the actor name from the list
                c1_cast.remove(c1)

#Creating a dictonary to get the couunts of each cast member
    c1_count = dict()
    for i in c1_cast:
            c1_count[i] = c1_count.get(i,0) + 1


        
#The same process is done for the next 4 actors the user has chosen

#ACTOR2
    c2_cast = []
    for i in directorcast:
        for j in i.values():
            if c2 in j:
                c2_cast.extend(j)
                c2_cast.remove(c2)
#ACTOR3
    c3_cast = []
    for i in directorcast:
        for j in i.values():
            if c3 in j:
                c3_cast.extend(j)
                c3_cast.remove(c3)
#ACTOR4
    c4_cast = []
    for i in directorcast:
        for j in i.values():
            if c4 in j:
                c4_cast.extend(j)
                c4_cast.remove(c4)
#ACTOR5
    c5_cast = []
    for i in directorcast:
        for j in i.values():
            if c5 in j:
                c5_cast.extend(j)
                c5_cast.remove(c5)


#Creating a Dictonary
    popular_subcast = {}
#Concatinating all sub cast who have worked with the 5 actors into one list
    combined_cast = c1_cast+c2_cast+c3_cast+c4_cast+c5_cast
    for each in combined_cast:
        if each not in popular_subcast:
            popular_subcast[each] = 1
        else:
            popular_subcast[each] += 1
 

    popular_subcast_sorted = []
# Sorting the data out as saving as a list again
    popular_subcast_sorted = sorted(popular_subcast.items(), key=operator.itemgetter(1),reverse = True)
    print(popular_subcast_sorted)
else:
    print('The actor name does not exist!')

Choose cast members for whom you want to calculate the similary score: 
Cast Member 1 : John Travolta
Cast Member 2 : Robert De Niro
Cast Member 3 : Brad Pitt
Cast Member 4 : Edward Norton
Cast Member 5 : Al Pacino
[('Harvey Keitel', 9), ('Jack Huston', 5), ('Aleksa Palladino', 4), ('Dascha Polanco', 4), ('Jake Hoffman', 4), ('Jesse Plemons', 4), ('Joe Pesci', 4), ('Kathrine Narducci', 4), ('Ray Romano', 4), ('Sebastian Maniscalco', 4), ('Stephanie Kurtzuba', 4), ('Steve Van Zandt', 4), ('Victor Argo', 3), ('Paul Herman', 3), ('Diane Keaton', 3), ('Robert Duvall', 3), ('Frank Sivero', 3), ('John Cazale', 3), ('Talia Shire', 3), ('Robert De Niro', 3), ('Albert Brooks', 2), ('Diahnne Abbott', 2), ('Frank Adu', 2), ('Garth Avery', 2), ('Gino Ardito', 2), ('Peter Boyle', 2), ('Al Pacino', 2), ('Anna Paquin', 2), ('Bobby Cannavale', 2), ('Stephen Graham', 2), ('Aly Mang', 2), ('Domenick Lombardozzi', 2), ('Gary Basaraba', 2), ('India Ennenga', 2), ('Joseph Russo', 2), ('Paul Ben-Victor', 2)

In [28]:
#Al Pacino, John Travolta, Robert De Niro, Brad Pitt, Edward Norton

In [29]:
#### SQL DB ATTEMPT
import sqlite3
import pandas as pd
#connections and cursor is create to connect and execute commands.
conn1 = sqlite3.connect('MovieInfoDatabase.sqlite')
c = conn1.cursor()
c.execute("""CREATE TABLE IF NOT EXISTS MovieInfoTable(
Movies VARCHAR,
Directors VARCHAR,
Casts VARCHAR)""")
#all_time_500.to_sql(name='MovieInfoTable', con=conn1, if_exists='replace', index=False)
conn1.commit()
conn1.close()
# fetch results from the table which was created and the data inserted to it
conn2 = sqlite3.connect('MovieInfoDatabase.db')
all_time_500 = pd.read_sql_query("SELECT * FROM MovieInfoTable", conn2)
conn2.commit()
conn2.close()

# END